In [ ]:
__author__ = 'stephen'

import caffe
import cv2, os
import numpy as np
import matplotlib.pyplot as plt
from lib.vdbc.dataset_factory import VDBC
from lib.vdbc.evaluate import Evaluator
from lib.vdbc.sample import gaussian_sample

from lib.data_layer.layer import get_next_mini_batch

PARAMS = (0.3, 0.3, 0.05, 0.7, 0.3)
IMS_PER_FRAME = 20


def vis_detection(im_path, gt, box):
    im = cv2.imread(im_path)[:, :, (2, 1, 0)]
    plt.cla()
    plt.imshow(im)
    # add ground-truth box
    plt.gca().add_patch(
        plt.Rectangle(
            (gt[0], gt[1]),
            gt[2], gt[3],
            fill=False,
            edgecolor='red',
            linewidth=1.5
        )
    )
    # add detection box
    plt.gca().add_patch(
        plt.Rectangle(
            (box[0], box[1]),
            box[2], box[3],
            fill=False,
            edgecolor='blue',
            linewidth=1.5
        )
    )

    plt.show()


def get_solver_net(train, test, weights):
    solver = caffe.SGDSolver(train)
    solver.net.copy_from(weights)

#     net = caffe.Net(test, caffe.TEST)
#     net.share_with(solver.net)

    return solver, net


def evaluate(evl, solver, net):
    # Initialize the net with the first frame
    im_path, gt = evl.init_frame()
    im = cv2.imread(im_path)
    samples = gaussian_sample(im, gt, PARAMS, 5500)
    db = []
    for i in range(len(samples)):
        db.append({
                'path': im_path,
                'img': im,
                'gt': gt,
                'samples': [samples[i]]
            })
    solver.net.layers[0].get_db(db)
    solver.step(1000)
    
    for i in range(2):
        im_path = evl.next_frame()
        im = cv2.imread(im_path)
        samples=gaussian_sample(im, gt, PARAMS, IMS_PER_FRAME)
        scores = np.zeros(IMS_PER_FRAME, dtype=np.float32)
        for i in range(IMS_PER_FRAME):
            db = [{
                'path': im_path,
                'img': im,
                'gt': gt,
                'samples': [samples[i]]
            }]
#             print samples[i]['box']
            blob = get_next_mini_batch(db)
            blob = {'data': blob['data'].astype(np.float32, copy=True)}
            net.blobs['data'].reshape(*blob['data'].shape)
            net.forward(data=blob['data'])
            print '~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'
            print net.blobs['cls_prob'].data
        

if __name__ == '__main__':
    caffe.set_mode_gpu()
    caffe.set_device(1)
    # get the deploy solver and net with pre-trained caffe model
    train = os.path.join('model', 'solver.prototxt')
    test = os.path.join('model', 'deploy_test.prototxt')
    weights = os.path.join('model', 'MDNet_iter_100000.caffemodel')

    solver, net = get_solver_net(train, test, weights)

    # get the Evaluator
#     dtype = 'VOT'
#     dbpath = os.path.join('data', 'vot2014')
#     gtpath = dbpath

#     vdbc = VDBC(dbtype=dtype, dbpath=dbpath, gtpath=gtpath, flush=True)

#     evl = Evaluator(vdbc)
#     evl.set_video(1)

#     evaluate(evl, solver, net)


In [1]:
print net.params['conv_3'][0].data

NameError: name 'net' is not defined

In [16]:
print net.params['fc4'][0].data

[[-0.00717333 -0.02337518 -0.02446882 ..., -0.01592515  0.01756274
   0.01766996]
 [ 0.01886117  0.02416935  0.00705118 ..., -0.02320821  0.01447972
   0.01288368]
 [-0.01867086  0.00476992  0.00599659 ...,  0.01353553  0.01880793
   0.00040074]
 ..., 
 [ 0.00082382  0.02208178  0.01332165 ..., -0.00154689 -0.01270521
   0.00579618]
 [ 0.01694149  0.01193865 -0.01946531 ...,  0.01368811  0.02144342
  -0.02395938]
 [ 0.02394009  0.01216421  0.02480587 ..., -0.00462591 -0.00302853
   0.01019591]]


In [18]:
print net.params['fc5'][0].data

0.0
